In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import bulk, streaming_bulk
import uuid
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}],timeout=30, max_retries=10, retry_on_timeout=True)
import json
import time
from pandas.io.json import json_normalize


In [ ]:
alldf = pd.DataFrame()

start = time.time()
page = es.search(
    index='alldataindex',
    scroll='40m',
    size=10000,
    _source=["id","content"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

alldf = alldf.append(json_normalize(page['hits']['hits']))
i=1
while (scroll_size > 0):
    #print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '40m')
    alldf = alldf.append(json_normalize(page['hits']['hits']))
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
end = time.time() - start
print(end)
print(len(alldf))


In [ ]:
nlpidf = pd.DataFrame()

start = time.time()
page = es.search(
    index='postnlpdata',
    scroll='40m',
    size=10000,
    _source=["NLPid","content"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

nlpidf = nlpidf.append(json_normalize(page['hits']['hits']))
i=1
while (scroll_size > 0):
    #print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '40m')
    nlpidf = nlpidf.append(json_normalize(page['hits']['hits']))
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
end = time.time() - start
print(end)
print(len(nlpidf))


In [ ]:
#Change column names : 
df.columns = ['content','id'] 


In [ ]:
#Delete columns: 
del df3['content'] 


In [ ]:
#Dataframe merge: 
df3=pd.merge(df, df4, how='inner', on='id') 